# Preguntas Internet of Things

In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://5fcc63a3e7cf:4042
SparkContext available as 'sc' (version = 3.2.1, master = local[*], app id = local-1655396568314)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@f54ec07


In [3]:
import spark.implicits._

import spark.implicits._


In [2]:
sc

res1: org.apache.spark.SparkContext = org.apache.spark.SparkContext@526376ab


## CASE CLASS

In [4]:
case class DeviceIoTData (battery_level: Long, c02_level: Long,
cca2: String, cca3: String, cn: String, device_id: Long,
device_name: String, humidity: Long, ip: String, latitude: Double,
lcd: String, longitude: Double, scale:String, temp: Long,
timestamp: Long)

defined class DeviceIoTData


In [5]:
val ds = spark.read
    .json("iot_devices.json")
    .as[DeviceIoTData]

ds: org.apache.spark.sql.Dataset[DeviceIoTData] = [battery_level: bigint, c02_level: bigint ... 13 more fields]


In [6]:
ds.show(3)

+-------------+---------+----+----+-------------+---------+--------------------+--------+-------------+--------+-----+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|           cn|device_id|         device_name|humidity|           ip|latitude|  lcd|longitude|  scale|temp|    timestamp|
+-------------+---------+----+----+-------------+---------+--------------------+--------+-------------+--------+-----+---------+-------+----+-------------+
|            8|      868|  US| USA|United States|        1|meter-gauge-1xbYRYcj|      51| 68.161.225.1|    38.0|green|    -97.0|Celsius|  34|1458444054093|
|            7|     1473|  NO| NOR|       Norway|        2|   sensor-pad-2n2Pea|      70|213.161.254.1|   62.47|  red|     6.15|Celsius|  11|1458444054119|
|            2|     1556|  IT| ITA|        Italy|        3| device-mac-36TWSKiT|      44|    88.36.5.1|   42.83|  red|    12.83|Celsius|  19|1458444054120|
+-------------+---------+----+----+-------------+---------+-----

In [7]:
val filterTempDS = ds.filter(d => {d.temp > 30 && d.humidity > 70})

filterTempDS: org.apache.spark.sql.Dataset[DeviceIoTData] = [battery_level: bigint, c02_level: bigint ... 13 more fields]


In [8]:
filterTempDS.show(5, false)

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 9) (5fcc63a3e7cf executor driver): java.lang.ClassCastException: class $iw cannot be cast to class $iw ($iw is in unnamed module of loader org.apache.spark.repl.ExecutorClassLoader @3a7bd559; $iw is in unnamed module of loader scala.tools.nsc.interpreter.IMain$TranslatingClassLoader @19c32eb7)

In [11]:
ds.select("*")
    .where($"temp">30 && $"humidity">70)
    .show(3)

+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+-----+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|           cn|device_id|         device_name|humidity|             ip|latitude|  lcd|longitude|  scale|temp|    timestamp|
+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+-----+---------+-------+----+-------------+
|            0|     1466|  US| USA|United States|       17|meter-gauge-17zb8...|      98|161.188.212.254|   39.95|  red|   -75.16|Celsius|  31|1458444054129|
|            9|      986|  FR| FRA|       France|       48|  sensor-pad-48jt4eL|      97|    90.37.208.1|   43.88|green|      4.9|Celsius|  31|1458444054151|
|            8|     1436|  US| USA|United States|       54|sensor-pad-5410CW...|      73|  204.15.64.249|   32.89|  red|  -117.13|Celsius|  34|1458444054155|
+-------------+---------+----+----+-------------+---

In [13]:
ds.select("temp","device_name","device_id","cca3")
    .where($"temp">25)
    .show(5)

+----+--------------------+---------+----+
|temp|         device_name|device_id|cca3|
+----+--------------------+---------+----+
|  34|meter-gauge-1xbYRYcj|        1| USA|
|  28|   sensor-pad-4mzWkz|        4| USA|
|  27|sensor-pad-6al7RT...|        6| USA|
|  27|sensor-pad-8xUD6p...|        8| JPN|
|  26|sensor-pad-10Bsyw...|       10| USA|
+----+--------------------+---------+----+
only showing top 5 rows



## Pregunta 1

In [14]:
ds.printSchema()

root
 |-- battery_level: long (nullable = true)
 |-- c02_level: long (nullable = true)
 |-- cca2: string (nullable = true)
 |-- cca3: string (nullable = true)
 |-- cn: string (nullable = true)
 |-- device_id: long (nullable = true)
 |-- device_name: string (nullable = true)
 |-- humidity: long (nullable = true)
 |-- ip: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- lcd: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- scale: string (nullable = true)
 |-- temp: long (nullable = true)
 |-- timestamp: long (nullable = true)



In [21]:
ds.select("battery_level","device_id")
    .where($"battery_level"<10 )
    .show(10)

+-------------+---------+
|battery_level|device_id|
+-------------+---------+
|            8|        1|
|            7|        2|
|            2|        3|
|            6|        4|
|            4|        5|
|            3|        6|
|            3|        7|
|            0|        8|
|            3|        9|
|            7|       10|
+-------------+---------+
only showing top 10 rows



In [27]:
ds.select("cn","c02_level")
    .groupBy("cn")
    .avg("c02_level")
    .withColumnRenamed("avg(c02_level)","media")
    .orderBy($"media" desc)
    .show(10)

+----------------+------------------+
|              cn|             media|
+----------------+------------------+
|           Gabon|            1523.0|
|Falkland Islands|            1424.0|
|          Monaco|            1421.5|
|          Kosovo|            1389.0|
|      San Marino|1379.6666666666667|
|         Liberia|            1374.5|
|           Syria|            1345.8|
|      Mauritania|1344.4285714285713|
|           Congo|          1333.375|
|           Tonga|            1323.0|
+----------------+------------------+
only showing top 10 rows



In [25]:
import org.apache.spark.sql.functions._

import org.apache.spark.sql.functions._
